In [ ]:
import ionpy
from ionpy.analysis import ResultsLoader
from ese.experiment.experiment import CalibrationExperiment
import seaborn as sns
import pandas as pd

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))
os.environ["CUDA_VISIBLE_DEVICES"] = '0,1,2,3' 

%load_ext yamlmagic
%load_ext autoreload
%autoreload 2

In [ ]:
# Results loader object does everything
rs = ResultsLoader()
root = "/storage/vbutoi/scratch/ESE"

In [ ]:
path = f"{root}/WMH_aug_runs"

dfc = rs.load_configs(
    path,
    properties=False,
)

df = rs.load_metrics(dfc)

In [ ]:
df

In [ ]:
best_exp = rs.get_experiment(
    df=df,
    exp_class=CalibrationExperiment,
    metric="val-dice_score",
    checkpoint="max-val-dice_score",
    device="cuda"
)

In [ ]:
%%yaml dataset_cfg 

data:
  _class: ese.experiment.datasets.WMH
  annotator: observer_o12
  axis: 0
  dataset: WMH  
  split: val
  slice_batch_size: 1
  task: Amsterdam 
  version: 0.2

In [ ]:
def get_dataset_perf(exp, split, dataset_config):

    dataset_cls = absolute_import(dataset_config.pop("_class"))
    dataset = data_cfg.pop("dataset")
    Dataset = dataset_cls(dataset=dataset, split=split, **dataset_config)
    dataloader = DataLoader(Dataset, shuffle=False, drop_last=False)

    items = []
    with torch.no_grad():
        for batch_idx, batch in enumerate(dataloader):
            x, y = to_device(batch, exp.device)
            yhat = exp.model(x)
            loss = exp.loss_func(yhat, y)
            items.append({
                "x": x.cpu().numpy(),
                "y": y.cpu().numpy(),
                "yhat": yhat.detach().cpu().numpy(),
                "loss": loss.item()
            })
    return items

In [ ]:
from ionpy.util import Config
#from ese.experiment.analysis import get_dataset_perf

# val_perf is a dict where each item is the subj id
# with the y, ypred, yloss, ydice
val_perf = get_dataset_perf(
    exp=best_exp, 
    spli="val",
    dataset_config=Config(dataset_cfg)
    )